In [4]:
import pandas as pd
import re
import spacy

# Load your dataset
df = pd.read_csv(r'C:\Users\Dell\Desktop\hasnain\Internship\dataset\test.csv')
df = df.dropna(subset=['article'])  # Assume 'article' column contains the long text


In [5]:
df.head()

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


In [6]:
# Clean text
def preprocess(text):
    text = re.sub(r'\s+', ' ', text)  # remove extra whitespace
    text = re.sub(r'\[[^]]*\]', '', text)  # remove brackets
    return text.strip()

df['clean_text'] = df['article'].apply(preprocess)


In [7]:
# Clean text
def preprocess(text):
    text = re.sub(r'\s+', ' ', text)  # remove extra whitespace
    text = re.sub(r'\[[^]]*\]', '', text)  # remove brackets
    return text.strip()

df['clean_text'] = df['article'].apply(preprocess)


In [24]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

nlp = spacy.load('en_core_web_sm')

def extractive_summary(text, per=0.3):
    doc = nlp(text)
    word_freq = {}
    for word in doc:
        if word.text.lower() not in STOP_WORDS and word.text.lower() not in punctuation:
            word_freq[word.text.lower()] = word_freq.get(word.text.lower(), 0) + 1

    max_freq = max(word_freq.values())
    for word in word_freq.keys():
        word_freq[word] = word_freq[word] / max_freq

    sent_scores = {}
    for sent in doc.sents:
        for word in sent:
            if word.text.lower() in word_freq:
                sent_scores[sent] = sent_scores.get(sent, 0) + word_freq[word.text.lower()]

    summary_sentences = nlargest(int(len(sent_scores) * per), sent_scores, key=sent_scores.get)
    return ' '.join([sent.text for sent in summary_sentences])

df['extractive_summary'] = df['clean_text'].apply(lambda x: extractive_summary(x))


In [31]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # Usually False on CPU-only machine


2.7.0+cpu
False


In [35]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

def extractive_summary(text, sentence_count=3):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LexRankSummarizer()
    summary = summarizer(parser.document, sentence_count)
    return ' '.join(str(sentence) for sentence in summary)

# Example usage
text = """Your long article or xyz text here..."""
print(extractive_summary(text, sentence_count=3))


Your long article or xyz text here...


In [38]:
print(df.columns)


Index(['id', 'article', 'highlights', 'clean_text', 'extractive_summary'], dtype='object')


In [43]:
if 'aarticle' in df.columns:
    print(df['aarticle'].iloc[0])
else:
    print("Column 'aarticle' does not exist in DataFrame!")


Column 'aarticle' does not exist in DataFrame!


In [47]:
print("\nArticle:\n", df['clean_text'].iloc[0])



Article:
 Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee. 'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting

In [48]:
if 'aarticle' in df.columns:
    print(df['aarticle'].iloc[0])
else:
    print("Column 'aarticle' does not exist!")


Column 'aarticle' does not exist!


In [53]:
# View example results
print("Original Text:\n", df['clean_text'].iloc[0])
print("\nExtractive Summary:\n", df['extractive_summary'].iloc[0])
print("\nArticle:\n", df['article'].iloc[0])


Original Text:
 Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee. 'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fig